In [1]:
import os
import json
from pprint import pprint


from google.cloud import translate_v3beta1 as translate

AUTH_JSON = 'LVL-MT-5e9bd9a5c96a.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = AUTH_JSON


client = translate.TranslationServiceClient()

with open(AUTH_JSON) as json_file:
    data = json.load(json_file)

project_id = data['project_id']
text = 'this is a test'
location = 'global'

parent = client.location_path(project_id, location)

# Varúð - þetta getur kostað

In [2]:
def google_translate_sentences(sentences, lang_from, lang_to, model):
    response = client.translate_text(
        model=f'projects/{project_id}/locations/global/models/general/{model}', # nmt or base
        parent=parent,
        contents=sentences,
        mime_type='text/plain',  # mime types: text/plain, text/html
        source_language_code=lang_from,
        target_language_code=lang_to)
    
    result = []
    for translation in response.translations:
        result.append(translation.translated_text)
    return result

In [32]:
with open('data/baseline-test.en') as en_test_file:
    result = []
    for line in en_test_file:
        result.append(line.strip())
pprint(result[:10])

["I 'm short of money .",
 '“ So he sulks and stays away ? ”',
 'in fact , the stars are so close that their surfaces overlap and a bridge '
 'has formed between them .',
 'in sufficiently congested regions like NGC 346 , with high levels of recent '
 'star birth , the result is a glorious , glowing vista for our telescopes to '
 'capture .',
 'there wax up her sons Thorleik and Bolli .',
 "on the other hand , when once got thoroughly into play , men 's brain would "
 'bear more work .',
 "do you like Mozart 's music ?",
 'we then compared these two ways to measure the strength of gravity - and the '
 'result was just what general relativity predicts , with an uncertainty of '
 'only 9 percent .',
 'now the cow went often down to the strand and the place : whereas the bale '
 'had been litten , and licked the stones on which the ashes thereof had been '
 'driven ;',
 'said Thorkel that his will would be that he should learn that soon .']


In [33]:
translated_base = google_translate_sentences(result[:10], 'en', 'is', 'base')
pprint(translated_base)

['Ég er stutt af peningum.',
 '"Svo sulks hann og helst í burtu? "',
 'í raun eru stjörnurnar svo nálægt að yfirborð þeirra skarast og brú hefur '
 'myndast á milli þeirra.',
 'í nægilega stíflaður svæðum eins og NGC 346, með mikið magn af nýlegum '
 'stjörnu fæðingu, niðurstaðan er glæsilega, glóandi sýn á sjónaukar okkar að '
 'handtaka.',
 'Það magnast upp synir hennar og Þorleikur.',
 "á hinn bóginn, þegar einu sinni fékk rækilega í leik, heili karla s 'bæri "
 'meiri vinnu.',
 "finnst þér eins og tónlist Mozart 's?",
 'við saman þá þessar tvær leiðir til að mæla styrk þyngdarafl - og '
 'niðurstaðan var bara það sem almenna afstæðiskenningin spáir, með óvissu um '
 'aðeins 9 prósent.',
 'Nú kýr fór oft niður til strandar og staður: en sem bagga hafði verið '
 'litten, og sleiktu steina sem askan hennar hafði verið ekið;',
 'sagði Þorkell að vilji hans væri að hann ætti að læra það fljótlega.']


In [35]:
translated_nmt = google_translate_sentences(result[:10], 'en', 'is', 'nmt')
pprint(translated_nmt)

['Mér vantar peninga.',
 '„Svo hann sulnar og heldur sig frá? “',
 'raunar eru stjörnurnar svo nálægt að yfirborð þeirra skarast og brú hefur '
 'myndast á milli þeirra.',
 'á nægilega þrengdum svæðum eins og NGC 346, með mikið magn nýlegra '
 'stjörnumyndunar, er niðurstaðan glæsileg, glóandi sýn fyrir sjónauka okkar '
 'til að fanga.',
 'þar vaxa upp synir hennar Þorleikur og Bolli.',
 'aftur á móti, þegar heilinn í karlmanni einu sinni komst rækilega í leik, þá '
 'myndi heili karla bera meiri vinnu.',
 'líkar þér tónlist Mozarts?',
 'við bárum svo saman þessar tvær leiðir til að mæla styrk þyngdaraflsins - og '
 'niðurstaðan var einmitt það sem almenn afstæðishyggja spáir, með aðeins 9 '
 'prósenta óvissu.',
 'nú fór kýrin oft niður að strandstaðnum og á staðinn. meðan búið var að '
 'kveikja á balanum og sleikja steinana, sem öskunni hafði verið ekið á;',
 'sagði Þorkel að vilji hans væri sá að hann ætti eftir að læra það fljótlega.']


In [36]:
with open('data/baseline-test.is') as en_test_file:
    target = []
    for line in en_test_file:
        target.append(line.strip())
pprint(target[:10])

['mig skortir fé .',
 '„ Svo að hann syrgir nú í fjarlægð og þorir ekki að koma ? “',
 'raunar eru þær svo þétt saman að yfirborð þeirra snertast og brú hefur '
 'myndast milli þeirra .',
 'á þéttum svæðum eins og NGC 346 , þar sem talsverð nýmyndun stjarna hefur '
 'orðið , verður þannig til myndrænt viðfangsefni sjónauka okkar .',
 'vaxa þar upp synir Guðrúnar , Þorleikur og Bolli .',
 'hins vegar mundi heili mannsins , þegar hann væri tekinn til starfa á annað '
 'borð , geta unnið meiri vinnu .',
 'finnst þér tónlist Mozarts góð ?',
 'við bárum svo saman þessar mælingar til að finna út styrk þyngdarkraftsins . '
 'niðurstöðurnar eru einmitt það sem almennt afstæði spáir fyrir um en óvissan '
 'er aðeins 9% .',
 'kýrin gekk oft ofan í fjöruna þar sem bálið hafði verið og sleikti steinana '
 'þar sem askan hafði fokið .',
 'Þorkell kvaðst það vilja mundu að hann vissi það brátt .']


In [4]:
with open('baseline_en-is/baseline-test-translated.is') as en_test_file:
    baseline = []
    for line in en_test_file:
        baseline.append(line.strip())
pprint(baseline[:10])

['mig skortir peninga .',
 '„ Svo sulks og verður hann í burtu ? “',
 'í fyrstu stjörnurnar eru svo þétt saman að yfirborð þeirra skarast sem og '
 'brú hefur myndast milli þeirra .',
 'þessar congested í svæðum eins og NGC 346 með hárri lítið við myndun stjarna '
 ', sem er í ævi stjörnu , glóandi sjónarsvið því sjónaukar safna til okkar .',
 'þar wax hana upp , Þorleikur og Bolli .',
 'á hinn bóginn var héruðunum , er stóð í heilanum leik , er menn skyldu bera '
 'meira .',
 'finnst þér tónlist Mozarts ?',
 'þessir tveir saman við þá til að mæla styrk þyngdarkraftsins . '
 'niðurstöðurnar var einmitt það sem almennt afstæði spáir fyrir um , eru '
 'aðeins fjögur prósent af 9 .',
 'nú kúna gekk oft ofan til strandar . og er þar höfðu verið litten báli og '
 'licked grjót á þá er þeir voru var rekinn .',
 'Þorkell mælti að hann mundi vera að hann skyldi að spyrja þig bráðlega .']
